In [1]:
import os 
from dotenv import find_dotenv, load_dotenv, dotenv_values
from langchain_openai import ChatOpenAI
import langchain_openai
import langchain
import langchain_groq
# from langchain.chat_models import ChatOpenAI
# from langchain.schema import HumanMessage
from typing import Union, List
import re
from langchain.agents import tool
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import Tool
from langchain.tools.render import render_text_description


# Load environment variables from a file
load_dotenv('../.env.secret')
#load_dotenv('.env.shared')
#langchain.debug = True


True

In [2]:

# Initialize the language model with specific parameters
llm_model = "gpt-3.5-turbo-0125"
model = ChatOpenAI(temperature=0)


In [3]:
# testing
@tool
def get_text_length(text: str) -> int:
    """Returns the length of a text by characters"""
    print(f"get_text_length enter with {text=}")
    # Clean the text by stripping unwanted characters
    text = text.strip("'\n").strip(
        '"'
    ) 

    return len(text)



def find_tool_by_name(tools: List[Tool], tool_name: str) -> Tool:
    for tool in tools:
        if tool.name == tool_name:
            return tool
    raise ValueError(f"Tool wtih name {tool_name} not found")

In [4]:
from langchain.agents.react.base import DocstoreExplorer
from langchain.docstore import Wikipedia

# Initialize DocstoreExplorer with Wikipedia as the document source
docstore = DocstoreExplorer(Wikipedia())


# Define available tools with their functionalities and descriptions
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]

print(tools)


[Tool(name='Search', description='useful for when you need to ask with search', func=<bound method DocstoreExplorer.search of <langchain.agents.react.base.DocstoreExplorer object at 0x000001E2477012A0>>), Tool(name='Lookup', description='useful for when you need to ask with lookup', func=<bound method DocstoreExplorer.lookup of <langchain.agents.react.base.DocstoreExplorer object at 0x000001E2477012A0>>)]


c:\Users\mhatr\anaconda3\envs\udemy\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0
  warn_deprecated(


In [5]:
# Define the prompt template for the ReAct agent
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

In [6]:
# Create a prompt template with tools and tool names
prompt = PromptTemplate.from_template(template=template).partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

print(prompt)

input_variables=['agent_scratchpad', 'input'] partial_variables={'tools': "Search(term: 'str') -> 'str' - useful for when you need to ask with search\nLookup(term: 'str') -> 'str' - useful for when you need to ask with lookup", 'tool_names': 'Search, Lookup'} template='\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought: {agent_scratchpad}\n'


In [7]:
# Initialize the language model with specific stop sequences
llm = ChatOpenAI(
        temperature=0,
        stop_sequences=["\nObservation", "Observation"]
    )

In [8]:
# Define the ReAct agent with input handling and output parsing
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["agent_scratchpad"]),
    }
    | prompt
    | llm
    | ReActSingleInputOutputParser()
)

In [11]:
# Process the agent's actions and handle responses
intermediate_steps = []
agent_step = ""
while not isinstance(agent_step, AgentFinish):
    input =  {
            "input": 'What profession does rohit sharma and virat kohli have in common?',
            "agent_scratchpad": intermediate_steps,
        }
    print('\n\n',input)
    agent_step: Union[AgentAction, AgentFinish] = agent.invoke(
       input
    )
    print(agent_step)

    if isinstance(agent_step, AgentAction):
        # Find the tool to use based on the agent's action
        tool_name = agent_step.tool
        tool_to_use = find_tool_by_name(tools, tool_name)
        tool_input = agent_step.tool_input
        
        # Execute the tool function and get the observation
        observation = tool_to_use.func(str(tool_input))
        print(f"{observation=}", '\n')
        intermediate_steps.append((agent_step, str(observation)))


# Print the final result once the agent finishes
if isinstance(agent_step, AgentFinish):
    print('\n\n', agent_step.return_values)



 {'input': 'What profession does rohit sharma and virat kohli have in common?', 'agent_scratchpad': []}
tool='Search' tool_input='Rohit Sharma profession' log='I should search for the professions of Rohit Sharma and Virat Kohli.\nAction: Search\nAction Input: "Rohit Sharma profession"'
observation="Could not find [Rohit Sharma profession]. Similar: ['Anushka Sharma', 'KL Rahul', 'Rohit Bohra', 'Singham', 'Shoaib Ali Bukhari', 'Jyotirmoy Dey', 'Desi Boyz', 'Dhadkan Zindaggi Kii', 'Kailash Hospital', 'Maddam Sir']" 



 {'input': 'What profession does rohit sharma and virat kohli have in common?', 'agent_scratchpad': [(AgentAction(tool='Search', tool_input='Rohit Sharma profession', log='I should search for the professions of Rohit Sharma and Virat Kohli.\nAction: Search\nAction Input: "Rohit Sharma profession"'), "Could not find [Rohit Sharma profession]. Similar: ['Anushka Sharma', 'KL Rahul', 'Rohit Bohra', 'Singham', 'Shoaib Ali Bukhari', 'Jyotirmoy Dey', 'Desi Boyz', 'Dhadkan Zind

In [21]:
print(agent_step)

return_values={'output': 'Rohit Sharma and Virat Kohli both have the profession of being international cricketers.'} log='I now know the final answer\nFinal Answer: Rohit Sharma and Virat Kohli both have the profession of being international cricketers.'


In [13]:
# code to show how tool information is passed to LLM for reasoning
print(render_text_description(tools))

Search(term: 'str') -> 'str' - useful for when you need to ask with search
Lookup(term: 'str') -> 'str' - useful for when you need to ask with lookup


In [14]:
# last thought of LLM
intermediate_steps[-1]

(AgentAction(tool='Search', tool_input='Virat Kohli profession', log='I should try searching for Virat Kohli\'s profession instead.\nAction: Search\nAction Input: "Virat Kohli profession"'),
 "Virat Kohli (Hindi pronunciation: [ʋɪˈɾɑːʈ ˈkoːɦli] ; born 5 November 1988) is an Indian international cricketer who plays Test and One Day International (ODI) cricket for the Indian national team. A former captain in all formats, Kohli retired from Twenty20 International (T20I) following India's win at the 2024 T20 World Cup. He is a right-handed batsman and an occasional unorthodox right arm quick bowler. He represents Royal Challengers Bengaluru in the Indian Premier League (IPL) and Delhi in domestic cricket. He holds the record as the highest run-scorer in  IPL, ranks third in T20I, third in ODI, and stands as the fourth-highest in international cricket. He also holds the record for scoring the most centuries in ODI cricket and stands second in the list of most international centuries scored

In [15]:
a = intermediate_steps[-1][0]
type(a), a

(langchain_core.agents.AgentAction,
 AgentAction(tool='Search', tool_input='Virat Kohli profession', log='I should try searching for Virat Kohli\'s profession instead.\nAction: Search\nAction Input: "Virat Kohli profession"'))

In [16]:
# code to show tool selection process internally
tool_name = a.tool
tool_to_use = find_tool_by_name(tools, tool_name)
tool_input = a.tool_input
tool_name, tool_to_use,tool_input

('Search',
 Tool(name='Search', description='useful for when you need to ask with search', func=<bound method DocstoreExplorer.search of <langchain.agents.react.base.DocstoreExplorer object at 0x000001E2477012A0>>),
 'Virat Kohli profession')

In [19]:
# check how tool used
print(tool_to_use.invoke(str(tool_input)))

Virat Kohli (Hindi pronunciation: [ʋɪˈɾɑːʈ ˈkoːɦli] ; born 5 November 1988) is an Indian international cricketer who plays Test and One Day International (ODI) cricket for the Indian national team. A former captain in all formats, Kohli retired from Twenty20 International (T20I) following India's win at the 2024 T20 World Cup. He is a right-handed batsman and an occasional unorthodox right arm quick bowler. He represents Royal Challengers Bengaluru in the Indian Premier League (IPL) and Delhi in domestic cricket. He holds the record as the highest run-scorer in  IPL, ranks third in T20I, third in ODI, and stands as the fourth-highest in international cricket. He also holds the record for scoring the most centuries in ODI cricket and stands second in the list of most international centuries scored. Kohli is widely regarded as one of the greatest batsmen of all time and the greatest batsman in the modern era. Kohli was a key member of the Indian team that won the 2011 Cricket World Cup, 